# Segmenting and clustering neighborhoods in city toronto 

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 1. Download and Explore Dataset

In [2]:
#import the library used to query a website
import urllib.request #if you are using python3+ version, import urllib.request

In [3]:
wiki ="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

### Query the website and return the html to the variable 'page'

In [4]:
page = urllib.request.urlopen(wiki) #For python 3 use urllib.request.urlopen(wiki)

### import the Beautiful soup functions to parse the data returned from the website

In [5]:
from bs4 import BeautifulSoup

### Parse the html in the 'page' variable, and store it in Beautiful Soup format

In [6]:
soup = BeautifulSoup(page)
#print(soup)

## Extract Table

In [18]:
all_tables=soup.find_all('table')
#print(all_tables)

### Find the right table

In [16]:
right_table=soup.find('table', class_='wikitable sortable')
#print(right_table)

### Extract the information to DataFrame

In [15]:
#Generate lists
A=[]
B=[]
C=[]
for row in right_table.findAll("tr"):
    cells = row.findAll('td')
    if len(cells)==3: #Only extract table body not heading
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
#print(A)
#print(B)
#print(C)

In [10]:
for n, i in enumerate(B):
    if i == 'Not assigned':
        B[n] = 'NaN'
#print(B)

In [20]:
for n, i in enumerate(C):
    if i == 'Not assigned\n':
        C[n] = 'NaN'
#print(C)

In [21]:
#Panda convert list to dataframe
d = {'Postcode': A, 'Borough': B, 'Neighbourhood': C}
df = pd.DataFrame(data=d)
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,NaN
9,M8A,Not assigned,NaN


### Ignore cells with a borough that is Not assigned

In [22]:
df = df[df.Borough != 'NaN']
df.head(5)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [23]:
#reset index
df.reset_index(drop=True, inplace=True)
df.head(7)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


### replace the neighbourhood with not assigned same as borough

In [24]:
import numpy as np
df['Neighbourhood'].replace('NaN', df['Borough'],inplace=True)
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
9,M8A,Not assigned,Not assigned


### group neighbourhood for same postcode area

In [48]:
df = df.groupby('Postcode').agg({'Borough':'first','Neighbourhood': ', '.join}).reset_index()
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood\n, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae\n
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park\n, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West\n"
9,M1N,Scarborough,"Birch Cliff, Cliffside West\n"


In [19]:
# print the number of rows of your dataframe.
df.shape

NameError: name 'df' is not defined